# **Reaction Explorer**
* **This is an example of reaction explorer. Script might be different according to how you structured your data directories**

In [1]:
from thermochemistry import Thermochemistry
from mkm.bag_of_energies import BagofEnergiesGen
import json
from pathlib import Path

/home/key2tw/.local/bin/miniconda3/envs/project-with-ocpmodel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* **Set vasp file located directory**

In [2]:
species_src = Path().cwd().parent.joinpath('example/species')
transitionstate_src = Path().cwd().parent.joinpath('example/transition_states')

In [3]:
beg = BagofEnergiesGen(species_src, transitionstate_src, temperature=[573.15])
bag_of_energies = beg.explorer()

* **Save bag of energy information as json or insert to mongoDB for your convenience**\
In this example, we are going to convert the data to json file 

In [4]:
json_dir = Path().cwd().parent.joinpath('example/info_bag.json')

with open(json_dir, 'w') as f:
    json.dump(bag_of_energies, f, indent=4)